In [4]:
# !pip install sklearn #pandas

     |████████████████████████████████| 6.8 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 7.8 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=a5159bf5cd2ec3d009adb3e617c3d20dafca6adfd80b28c1d4562d145b3d7dd2
  Stored in directory: /root/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
# import pandas as pd
import os
import sys

from env.TFenv_masked import HangmanEnvironment

from tf_agents.environments import utils
from tf_agents.environments import tf_py_environment

from tf_agents.agents.dqn import dqn_agent
from tf_agents.networks import q_network
from tf_agents.utils import common

from learningClass import learningHelper



tf.random.set_seed(12)
tf.print(tf.config.list_physical_devices('GPU') )
tf.compat.v1.enable_v2_behavior()

[]


## Read OCHLV data from the file

In [2]:
# df = pd.read_csv('test.csv')
# df = df.drop(['Date','Adj Close'], axis=1)
# train_index = 200
# df_train = df.iloc[:train_index]
# df_test = df.iloc[train_index:]
# df_train.head(2)

,High,Low,Open,Close,Volume
0,111.440002,107.349998,111.389999,109.330002,53204600.0
1,108.650002,105.410004,108.290001,106.250000,64285500.0


## Create and validate enviroment

In [2]:
dictionary_path = "../../../properties/words_250000_train.txt"
reward_map = {
    "game_success_reward": 1.00,
    "lose_reward": -0.,
    "guess_success_reward": 0.2,
    "guess_fail_reward": -0.,
    "guess_repeat_reward": -5.,
}
life_initial = 15


environment = HangmanEnvironment(dictionary_path, reward_map,life_initial)
utils.validate_py_environment(environment, episodes=2)

print('action_spec:', environment.action_spec())
print('time_step_spec.observation:', environment.time_step_spec().observation)
print('time_step_spec.step_type:', environment.time_step_spec().step_type)
print('time_step_spec.discount:', environment.time_step_spec().discount)
print('time_step_spec.reward:', environment.time_step_spec().reward)



action_spec: BoundedArraySpec(shape=(), dtype=dtype('int32'), name='letter', minimum=0, maximum=25)
time_step_spec.observation: {'observations': BoundedArraySpec(shape=(1, 30), dtype=dtype('float32'), name='game', minimum=-1.0, maximum=26.0), 'legal_moves': ArraySpec(shape=(26,), dtype=dtype('float32'), name=None)}
time_step_spec.step_type: ArraySpec(shape=(), dtype=dtype('int32'), name='step_type')
time_step_spec.discount: BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0)
time_step_spec.reward: ArraySpec(shape=(), dtype=dtype('float32'), name='reward')


## Create traning and validation enviroment

In [3]:
train_env = tf_py_environment.TFPyEnvironment(HangmanEnvironment(dictionary_path, reward_map,life_initial))
eval_env = tf_py_environment.TFPyEnvironment(HangmanEnvironment(dictionary_path, reward_map,life_initial))

## Define trainig paramters

In [5]:
learning_rate = 1e-3  

## Create q_network

In [7]:
#network configuration
fc_layer_params = (40,)

# as we are using dictionary in our enviroment, we will create preprocessing layer
hidden_layers = (512, 512)
preprocessing_layers = {
    'observations': tf.keras.layers.Flatten(),
    'legal_moves': tf.keras.layers.Dense(26,)
    }
preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)
def observation_and_action_constraint_splitter(obs):
	return obs['observations'], np.logical_not(obs['legal_moves'])

#create a q_net
q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    preprocessing_layers=preprocessing_layers,
    preprocessing_combiner=preprocessing_combiner,
    fc_layer_params=fc_layer_params)

## Create the DQN-agent

In [8]:
#create optimizer
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

#create a global step coubter
#train_step_counter = tf.Variable(0)
global_step = tf.compat.v1.train.get_or_create_global_step()

#create agent
agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter,
    #train_step_counter=train_step_counter)
    train_step_counter=global_step)

agent.initialize()

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

In [9]:
magent = learningHelper(train_env=train_env, test_env=eval_env, agent=agent, global_step=global_step, collect_episodes = 10,
eval_interval=5, verbose=0, batch_size=64, chkpdir='./fc_chkp/')
magent.restore_check_point()

selected agent collect_policy


NotImplementedError: Cannot convert a symbolic Tensor (observation/legal_moves:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [13]:
%%time
magent.train_agent(1)
#magent.train_agent_with_avg_ret_condition(100, 10000, 100)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


Global steps 701: Traning Loss 28282.611328125
CPU times: user 2.82 s, sys: 119 ms, total: 2.94 s
Wall time: 2.69 s


In [11]:
for _ in range(8):
    magent.evaluate_agent(80)


eval episodes = 80: Average Return = -47.49997329711914
eval episodes = 80: Average Return = 175.19985961914062
eval episodes = 80: Average Return = 50.70004653930664
eval episodes = 80: Average Return = -218.29971313476562
eval episodes = 80: Average Return = -247.70010375976562
eval episodes = 80: Average Return = -287.9000244140625
eval episodes = 80: Average Return = -118.69987487792969
eval episodes = 80: Average Return = -410.4000549316406


In [37]:
magent.store_check_point()

In [16]:
magent.restore_check_point()